### **This notebook is used to make the UNet.py script which will run of the server side to generate segmentation results on web-interface**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pip install nilearn

     |████████████████████████████████| 3.1MB 8.6MB/s 


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
import keras
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from sklearn.model_selection import train_test_split
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
from keras import backend as K
import glob
import imageio
import skimage.io as io
import skimage.color as color
import random as r
import cv2
import math
from nilearn import plotting
from PIL import Image
import pickle
import skimage.transform as skTrans
from nilearn import image
from nilearn.image import resample_img
import nibabel.processing
import warnings

/usr/local/lib/python3.7/dist-packages/nilearn/datasets/__init__.py:90: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


### **CreateModel() function**

In [ ]:
"""
rescale_Nii(nifti_file):
This function takes a .nii files as an input and rescales it according to the 
values of voxel_dims by creating a new affine transform. The new affine transform
is used in resample_img() function from nibable library which will transform the images
accordinly inputs for resample_img() are tagert image, target affine transform and target shape

Input:
nifti_file: A .nii file which we want to rescale

Output:
rescaled .nii file with dimensions as defined in target_shape
"""
def rescale_Nii(nifti_file):
    warnings.filterwarnings("ignore")
    img=nifti_file
    #voxel_dims=[3.8, 3.8,1]
    voxel_dims=[1.60, 1.60,1]
    

    # downl sample to 128*128*155
    #target_shape=(64,64,130)
    target_shape=(128,128,155)
    # Initialize target_affine
    target_affine = img.affine.copy()
    # Calculate the translation part of the affine
    spatial_dimensions = (img.header['dim'] * img.header['pixdim'])[1:4]
    
    # Calculate the translation affine as a proportion of the real world
    # spatial dimensions
    image_center_as_prop = img.affine[0:3,3] / spatial_dimensions
    
    # Calculate the equivalent center coordinates in the target image
    dimensions_of_target_image = (np.array(voxel_dims) * np.array(target_shape))
    target_center_coords =  dimensions_of_target_image * image_center_as_prop
    # Decompose the image affine to allow scaling
    u,s,v = np.linalg.svd(target_affine[:3,:3],full_matrices=False)
    
    # Rescale the image to the appropriate voxel dimensions
    s = voxel_dims
    
    # Reconstruct the affine
    target_affine[:3,:3] = u @ np.diag(s) @ v

    # Set the translation component of the affine computed from the input
    target_affine[:3,3] = target_center_coords 
  

    #target_affine = rescale_affine(target_affine,voxel_dims,target_center_coords)
    resampled_img = resample_img(img, target_affine=target_affine,target_shape=target_shape)
    resampled_img.header.set_zooms((np.absolute(voxel_dims)))
    return resampled_img

In [ ]:
def Convolution(input_tensor,filters):
    
    x = Conv2D(filters=filters,kernel_size=(3, 3),padding = 'same',strides=(1, 1))(input_tensor)
    x = BatchNormalization()(x)
    x = Activation('relu')(x) 
    return x

def model(input_shape):
    
    inputs = Input((input_shape))
    
    conv_1 = Convolution(inputs,32)
    maxp_1 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_1)
    
    conv_2 = Convolution(maxp_1,64)
    maxp_2 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_2)
    
    conv_3 = Convolution(maxp_2,128)
    maxp_3 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_3)
    
    conv_4 = Convolution(maxp_3,256)
    maxp_4 = MaxPooling2D(pool_size = (2, 2), strides = (2, 2), padding = 'same') (conv_4)
    
    conv_5 = Convolution(maxp_4,512)
    upsample_6 = UpSampling2D((2, 2)) (conv_5)
    
    conv_6 = Convolution(upsample_6,256)
    upsample_7 = UpSampling2D((2, 2)) (conv_6)
    
    upsample_7 = concatenate([upsample_7, conv_3])
    
    conv_7 = Convolution(upsample_7,128)
    upsample_8 = UpSampling2D((2, 2)) (conv_7)
    
    conv_8 = Convolution(upsample_8,64)
    upsample_9 = UpSampling2D((2, 2)) (conv_8)
    
    upsample_9 = concatenate([upsample_9, conv_1])
    
    conv_9 = Convolution(upsample_9,32)
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (conv_9)
    
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model

In [ ]:
# Loding the Light weighted CNN
model = model(input_shape = (128,128,1))


In [ ]:
model.load_weights('/content/drive/MyDrive/MRI Data/Model_History/BraTs2020_(Fused_0-369).h5')

### **TumorPredict(4- array for paths of files,1-location to store the output ) function**

In [ ]:
t1=('/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t1.nii')
t1ce=('//content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t1ce.nii')
flair=('/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_flair.nii')
t2=('/content/drive/MyDrive/MRI Data/BraTS2020_ValidationData/MICCAI_BraTS2020_ValidationData/BraTS20_Validation_001/BraTS20_Validation_001_t2.nii')

In [ ]:
modalities=[t1,t1ce,flair,t2]

In [ ]:
all_modalities = []    
for modality in modalities:      
        nifti_file   = nib.load(modality)
        nifti_file= rescale_Nii(nifti_file)
        brain_numpy  = np.asarray(nifti_file.dataobj)    
        all_modalities.append(brain_numpy)
brain_affine   = nifti_file.affine
all_modalities = np.array(all_modalities)
all_modalities = np.rint(all_modalities).astype(np.int16)
all_modalities = all_modalities[:, :, :, :]
all_modalities = np.transpose(all_modalities)
avg_modality=[]
for i in range(len(all_modalities)):
    x=(all_modalities[i,:,:,0]+all_modalities[i,:,:,1]+all_modalities[i,:,:,2]+all_modalities[i,:,:,3])/4
    avg_modality.append(x)  
avg_modality=np.array(avg_modality)  
 


In [ ]:
TR=np.array(avg_modality[:,:,:],dtype='float32')

In [ ]:
pref_Tumor = model.predict(TR)

### **SaveOutput(location to save,pref_tumor) function.**

In [ ]:
# This part save the pred_Tumor array into the given file directory for each silce as a .png image
for i in range(len(pref_Tumor)):
  fig = plt.figure(figsize=(5,5))
  plt.title('Slice Number:'+str(i+1))
  plt.imshow(np.squeeze(TR[i,:,:]),cmap='gray')
  plt.imshow(np.squeeze(pref_Tumor[i,:,:,0]),alpha=0.3,cmap='Reds')
  plt.savefig('/content/drive/MyDrive/MRI Data/Output/Slice Number: '+str(i+1).zfill(3)+'.png')
  

In [ ]:
#Output the .nii file
x=pref_Tumor[:,:,:,0]
x = np.transpose(x, (1,2,0))
img = nib.Nifti1Image(x, brain_affine)
nib.save(img, '/content/drive/MyDrive/MRI Data/Output/Segmentation.nii.gz') 